In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [2]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [3]:
#Get Teams Names and ids
def getTeams():
    driver.get('https://www.whoscored.com/Regions/252/Tournaments/2/')
    base = driver.find_element_by_id('standings-16368-content')
    rows =base.find_elements_by_tag_name('tr')
    teams = []
    for item in range(0,len(rows)):
        data = rows[item].find_elements_by_tag_name('td')
        teams.append(data[1].text)
    links = []
    for i in range(0,len(rows)):
        links.append(rows[i].find_elements_by_class_name('team-link')[0].get_attribute('href'))
    team_ids = []
    for i in range(0,len(links)):
        team_ids.append(re.search(r'\d+', links[i]).group())
    team_ids = list(map(int, team_ids))
    for i in range(0,len(teams)):
        if ' ' in teams[i]:
            teams[i] = teams[i].replace(' ','-')

    teams = dict(zip(teams, team_ids))
    return teams
teams = getTeams()

In [4]:
# # Get Players stats for each team.
def getPlayerSummary(teams):
    values =[]
    dfOut=[]
    for i in teams.values(): 
        driver.get('https://www.whoscored.com/Teams/'+str(i))
        driver.implicitly_wait(40)
        base = driver.find_element_by_id('top-player-stats-summary-grid')
        table = base.find_element_by_id('player-table-statistics-body')
        tableRows = table.find_elements_by_tag_name('tr')
        for item in range(1,len(tableRows)):
                row     = tableRows[item].find_elements_by_tag_name('td')
                values.append([row[i].text for i in range(1,len(row))])
        for j in range(0,len(values)):
            dfOut.append([i,values[j][0],values[j][1],values[j][2],values[j][3],values[j][4],
                          values[j][5],values[j][6],
                                values[j][7],values[j][8],values[j][9],values[j][10],values[j][11],
                         values[j][12],values[j][13],values[j][14]])
    dfOut = pd.DataFrame(dfOut)
    dfOut = dfOut.drop_duplicates()
    return dfOut
players_summary = getPlayerSummary(teams)

In [116]:
driver.get('https://www.whoscored.com/Teams/'+str(23))
driver.implicitly_wait(20)
base = driver.find_element_by_id('top-player-stats-summary-grid')

In [6]:
player_summary_df = pd.DataFrame(columns=["R", "player", "height", "weight", "apps",
                                          'mins','goals','Assists','Yel','Red',
                                          'SpG','PS%','AerialsWon','MotM','Rating'])

In [50]:
driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
driver.implicitly_wait(20)
driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[1].click()
base = driver.find_elements_by_id('statistics-table-defensive')
tableRows = base[0].find_elements_by_tag_name('tr')
tableRows[1].text

In [59]:
driver.get('https://www.whoscored.com/Regions/252/Tournaments/2/')
base = driver.find_element_by_id('standings-16368-content')
rows =base.find_elements_by_tag_name('tr')